In [96]:
from datasets import load_dataset
ashaar = load_dataset('Zaid/ashaar_datasetv2')

Found cached dataset parquet (/home/alyafey22/.cache/huggingface/datasets/Zaid___parquet/Zaid--ashaar_datasetv2-029803fd0fe2f567/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [97]:
ashaar['train'][0]['text']

'<|meter_0|> م <|theme_18|> <|psep|> <|bsep|> أَصبَحَ المُلك لِلَّذي فَطر الخَل <|vsep|> قَ بِتَقديرٍ للعَزيز العَليمِ </|bsep|> <|bsep|> غافر الذَنب للمسيءِ بِعَفوٍ <|vsep|> قابل التَوب ذي العَطاء العَميمِ </|bsep|> <|bsep|> مُرسل المُصطَفى البَشير ِلَينا <|vsep|> رَحمة مِنهُ بِالكَلام القَديمِ </|bsep|> <|bsep|> رَبَنا رَبّنا ِلَيكَ أَنينا <|vsep|> فَأَجرنا مِن حَر نار الجَحيمِ </|bsep|> <|bsep|> وَاكفِنا شَرّ ما نَخاف بِلُطفٍ <|vsep|> يا عَظيماً يَرجى لِكُل عَظيمِ </|bsep|> <|bsep|> وَتَقبل أَعمالَنا وَاعفُ عَنا <|vsep|> وَأَنلنا دُخول دار النَعيمِ </|bsep|> <|bsep|> بِنَبي بَعثَتهُ فَهَدانا <|vsep|> لِصِراط مِن الهُدى مُستَقيمِ </|bsep|> <|bsep|> وَبِمَن نَحنُ في حِماهُ مَدى الدَهر <|vsep|> أَخيهِ يَحيى الحصور الكَريمِ </|bsep|> <|bsep|> أَدرك أَدرك قَوماً أَتوا بافتقار <|vsep|> وَاِنكِسار وَمَدمَع مَسجومِ </|bsep|> </|psep|>'

In [98]:
ashaar

DatasetDict({
    train: Dataset({
        features: ['poem title', 'poem meter', 'poem verses', 'poem theme', 'poem url', 'poet name', 'poet description', 'poet url', 'poet era', 'poet location', 'poem description', 'poem language type', 'text'],
        num_rows: 212499
    })
})

In [106]:
import json 
theme_to_token = json.load(open("theme_tokens.json", "r"))
token_to_theme = {t:m for m,t in theme_to_token.items()}

meter_to_token = json.load(open("meter_tokens.json", "r"))
token_to_meter = {t:m for m,t in meter_to_token.items()}

In [107]:
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt_tokenizer = AutoTokenizer.from_pretrained('Zaid/ashaar_tokenizerv2')
encoded_input = gpt_tokenizer(['<|psep|>'], return_tensors='pt')
model = AutoModelForCausalLM.from_pretrained('gpt_results/gpt_results_fix_tokens_qafiyah_theme/checkpoint-50000')

In [139]:
theme = "قصيدة دينية"
meter = "الكامل"
qafiyah = "ك"
prompt = f"{meter_to_token[meter]} {qafiyah} {theme_to_token[theme]}"
print(prompt)
encoded_input = gpt_tokenizer(prompt, return_tensors='pt')
output = model.generate(**encoded_input, max_length = 512, top_p = 3, do_sample=True)
print(encoded_input)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<|meter_14|> ك <|theme_18|>
{'input_ids': tensor([[ 21, 105,  53,  42]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [140]:
result = ""
prev_token = ""

for i, beam in enumerate(output[:, len(encoded_input.input_ids[0]):]):
    for token in beam:
        decoded = gpt_tokenizer.decode(token)
        if 'meter' in decoded or 'theme' in decoded:
            break
        if decoded == "<|vsep|>":
            result += "*** "
        elif decoded in ["<|bsep|>", "</|bsep|>"]:
            result += "\n"
        elif decoded in ['<|psep|>', '</|psep|>']:
            pass
        else:
            result += decoded
        prev_token = decoded
    else:
        break
print(theme+" "+ f"من بحر {meter} مع قافية بحر ({qafiyah})" + "\n" +result)

قصيدة دينية من بحر الكامل مع قافية بحر (ك)

أَدَبٌ غَدَت عَبراً لَها *** تَقبيلَ طِفلٍ في لَظاك 

راعَت مَعاهِدُ لَيلَتي *** في حَيثُ لاتَكَ في هَناك 

سِفرَ استَرَقَّ الدَمعُ يُح *** رِقُ ثَديُها بَعدَ السِفاك 

في غَفلَةٍ زُهِيَت بِمِص *** بَح عَنهُمُ يَومَ الصَباك 

وَغَدَت سَوافِرُ بَعدَما *** سَفَرَت بِنَستُرِ بِالنِساك 

وَنَكَثتُ وَهيَ حَبيبَةٌ *** يَومَ التَنَصُّلِ وَالشَراك 

تَجِدُ اللُبانَةَ فَالأُنو *** فَ تَلَبُّساً وَثَنى الرِكاك 

وَالوَبلُ في شَوقٍ لِِم *** لاقَ حَبيبَهُمُ وَحاك 

